In [ ]:
import pandas as pd


# Import the necessary libraries
import os
from openai import OpenAI

client = OpenAI(api_key = api2)

def prompt(prompt, m, s, t, results, temp=1):

    #results = {'prompt':[], 'response': [], 'model': [], 'temperature': [], 'seed':[], 'NEM': []}
    try:
        response = client.chat.completions.create(
            model = m,
            seed = s,
            temperature = temp,
            messages=[
                {"role": "system", "content": "You are ChatGPT."},
                {"role": "user", "content": prompt}
            ],
            max_completion_tokens=100000,  # Adjusted to a reasonable length for 10 sentences
        )
    except openai.OpenAIError as e:
        print(f"OpenAI API error: {e}")

    a = response.choices[0].message.content.strip()

    results['prompt'].append(prompt)
    results['temperature'].append(temp)
    results['response'].append(a)
    results['model'].append(m)
    results['seed'].append(s)
    results['text'].append(t)


    return results



In [ ]:

import pandas as pd

languages = ['chinese']
seeds = [0,5]
#a.head()

pr = """你能否判断文本中位于 [PET_BOUNDARY] 分界符之间的短语是否以委婉的方式使用？请不要在回答中包含前言或结尾。仅输出数字 1 表示其为委婉，0 表示不是。

以下是一些文本及其相应的输出：
Text: '前年，父亲因病[PET_BOUNDARY]去世[PET_BOUNDARY]，一家人陷入沉重的思念。', output 1.
Text: '新德里消息：印度一列火车20日上午发生出轨事故，当地媒体称至少已导致15人[PET_BOUNDARY]遇难[PET_BOUNDARY]，另有逾40名乘客受伤。 印度官方随后展开救援工作，并宣布向遇难者家属发放抚恤金。', output 1.
Text: '分手三个月了，还是很想他。 射手遇到白羊，整个自我就[PET_BOUNDARY]没了[PET_BOUNDARY]。 想从女汉子变成软妹子。', output 0.
Text: '拿我来说吧，我宁愿早起也不会熬夜。 如果注定今天只能睡4个小时，我会在23点之前[PET_BOUNDARY]上床[PET_BOUNDARY]，定好闹钟，在凌晨起来继续奋斗。 因为，子夜时分是人体自我更新和修复最活跃的时刻，抓住这段时间至关重要。', output 0.

这是文本： '
"""
for x in languages:
    for y in range(seeds[0],seeds[1]):


        a = pd.read_csv(x+' euphs 2025-09-17/test_0_'+x+'.csv')
        results = {'prompt':[], 'text': [], 'response': [], 'model': [], 'temperature': [], 'seed':[]}

        for i in a['TEXT']:
            prompt(pr + i +"'", 'gpt-5-nano', y, i, results, 1)
        results = pd.DataFrame(results)
        results['true_value'] = a['LABEL']
        results.to_csv('fewshot_prompt2_'+x+'_'+str(y)+'.csv')
